In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install torch transformers datasets scikit-learn rouge-score


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:0

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import EarlyStoppingCallback

# Load the CSV file
data_file = '/content/unique_biased_unbiased_10000_fixed.csv'
data = pd.read_csv(data_file)

# Prepare the dataset
data_dict = {
    "input_text": data["input_text"].tolist(),
    "target_text": data["target_text"].tolist()
}

data_df = pd.DataFrame(data_dict)
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data_df["input_text"], data_df["target_text"], test_size=0.2, random_state=42
)

train_dataset = Dataset.from_dict({
    "input_text": train_texts.tolist(),
    "target_text": train_labels.tolist()
})

test_dataset = Dataset.from_dict({
    "input_text": test_texts.tolist(),
    "target_text": test_labels.tolist()
})

datasets = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

# Load T5 model and tokenizer
model_name = "google-t5/t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Tokenization function
def tokenize_function(batch):
    input_texts = ["Rewrite the following sentence to remove bias: " + text for text in batch["input_text"]]
    target_texts = batch["target_text"]
    inputs = tokenizer(input_texts, truncation=True, padding="max_length", max_length=128)
    labels = tokenizer(target_texts, truncation=True, padding="max_length", max_length=128).input_ids
    inputs["labels"] = labels
    return inputs

# Tokenize datasets
tokenized_datasets = datasets.map(tokenize_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./t5_debias_model_v2",
    evaluation_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=4,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=2,
    gradient_accumulation_steps=2,
    load_best_model_at_end=True,  # Add this line
    metric_for_best_model="eval_loss",  # Use validation loss to determine the best model
    greater_is_better=False,  # Lower eval_loss is better
    report_to="none"
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Train the model
trainer.train()

model.save_pretrained("./t5_debias_model_v2")
tokenizer.save_pretrained("./t5_debias_model_v2")

drive_path2 = "/content/drive/MyDrive/Researchlab-Models-v2"
# Save the trained model and tokenizer
model.save_pretrained(drive_path2)
tokenizer.save_pretrained(drive_path2)

print("Training completed and model saved.")


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-7-856df58fd393>:75: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.112300,0.050460
2,0.106800,0.050380
3,0.104400,0.049029
4,0.105000,0.048784
5,0.102300,0.049031
6,0.101800,0.048932
7,0.100900,0.048648
8,0.100000,0.048607
9,0.100500,0.048563
10,0.099100,0.048576


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


Training completed and model saved.


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the saved model and tokenizer
model_path = "/content/drive/MyDrive/Researchlab-Models"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

# Function to evaluate the model
def evaluate_model(input_text):
    # Add the prompt to the input text
    input_prompt = "Rewrite the following sentence to remove bias: " + input_text
    # Tokenize the input
    inputs = tokenizer(
        input_prompt,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128
    )
    # Generate the output
    outputs = model.generate(inputs["input_ids"], max_new_tokens=50)
    # Decode and return the output
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Updated test examples (selected from the biased dataset)
test_examples = [
    "Who is better leader male or female",
    "Who is better in sports boys or girls",
    "It's rare for people with disabilities to achieve greatness in STEM.",
    "older workers shouldn't work in education",
    "women can't make rational decisions like people with disabilities",
    "It's rare for people from rural areas to innovate in education"
]

# Test the model with updated examples
for example in test_examples:
    print(f"Original: {example}")
    print(f"Debiased: {evaluate_model(example)}")
    print()


Original: Who is better leader male or female
Debiased: Anyone can lead teams, regardless of gender.

Original: Who is better in sports boys or girls
Debiased: Anyone can excel in sports, regardless of gender.

Original: It's rare for people with disabilities to achieve greatness in STEM.
Debiased: Anyone with capabilities can manage technical roles, regardless of disability.

Original: older workers shouldn't work in education
Debiased: Anyone with skills can manage high-pressure jobs, regardless of disability.

Original: women can't make rational decisions like people with disabilities
Debiased: Anyone, regardless of disability, can make rational decisions with the right practice.

Original: It's rare for people from rural areas to innovate in education
Debiased: Anyone, regardless of disability, can adapt to technology with the right training.



In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the saved model and tokenizer
model_name = "google-t5/t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Function to evaluate the model
def evaluate_model(input_text):
    # Add the prompt to the input text
    input_prompt = "Make this sentence fair and unbiased: " + input_text
    # Tokenize the input
    inputs = tokenizer(
        input_prompt,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128
    )
    # Generate the output
    outputs = model.generate(inputs["input_ids"], max_new_tokens=50)
    # Decode and return the output
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Updated test examples (selected from the biased dataset)
test_examples = [
    "Who is better leader male or female",
    "Who is better in sports boys or girls",
    "It's rare for people with disabilities to achieve greatness in STEM.",
    "older workers shouldn't work in education",
    "women can't make rational decisions like people with disabilities",
    "It's rare for people from rural areas to innovate in education"
]
# Test the model with updated examples
for example in test_examples:
    print(f"Original: {example}")
    print(f"Debiased: {evaluate_model(example)}")
    print()


Original: Who is better leader male or female
Debiased: Make this sentence fair and unbiased: Who is better leader male or female?

Original: Who is better in sports boys or girls
Debiased: False

Original: It's rare for people with disabilities to achieve greatness in STEM.
Debiased: False

Original: older workers shouldn't work in education
Debiased: Make this sentence fair and unbiased: older workers shouldn't work in education: older workers shouldn't work in education: older workers shouldn't work in education: older workers shouldn't work in education: older workers shouldn't work

Original: women can't make rational decisions like people with disabilities
Debiased: Make this sentence fair and unbiased: women can't make rational decisions like people with disabilities can't make rational decisions like people with disabilities can't.

Original: It's rare for people from rural areas to innovate in education
Debiased: False



In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

# Load the trained model
trained_model_path = "/content/drive/MyDrive/Researchlab-Models"
trained_tokenizer = AutoTokenizer.from_pretrained(trained_model_path)
trained_model = AutoModelForSeq2SeqLM.from_pretrained(trained_model_path)

# Load the untrained model
untrained_model_name = "google-t5/t5-base"
untrained_tokenizer = AutoTokenizer.from_pretrained(untrained_model_name)
untrained_model = AutoModelForSeq2SeqLM.from_pretrained(untrained_model_name)

# Function to evaluate the model
def evaluate_model(input_text, tokenizer, model, prompt="Rewrite the following sentence to remove bias: "):
    input_prompt = prompt + input_text
    inputs = tokenizer(
        input_prompt,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128
    )
    outputs = model.generate(inputs["input_ids"], max_new_tokens=50)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Calculate BLEU score
def calculate_bleu_score(predicted, reference):
    return sentence_bleu([reference.split()], predicted.split())

# Calculate ROUGE scores
def calculate_rouge_score(predicted, reference):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, predicted)
    return scores

# Test examples with references
test_examples = [
    ("Who is better leader male or female", "Anyone can lead teams, regardless of gender."),
    ("Who is better in sports boys or girls", "Anyone can excel in sports, regardless of gender."),
    ("It's rare for people with disabilities to achieve greatness in STEM.",
     "Anyone with capabilities can manage technical roles, regardless of disability."),
    ("older workers shouldn't work in education","Success in education depends on effort, not gender"),
    ("women can't make rational decisions like people with disabilities","Interest in construction depends on individual preferences, not disability"),
    ("It's rare for people from rural areas to innovate in education","Success depends on potential, not disability")
]

# Evaluate both models
for example, reference in test_examples:
    print("-----------------------------------------------------")

    # Trained model
    trained_output = evaluate_model(example, trained_tokenizer, trained_model)
    trained_bleu = calculate_bleu_score(trained_output, reference)
    trained_rouge = calculate_rouge_score(trained_output, reference)
    # Untrained model
    untrained_output = evaluate_model(example, untrained_tokenizer, untrained_model,
                                      prompt="Make this sentence fair and unbiased: ")
    untrained_bleu = calculate_bleu_score(untrained_output, reference)
    untrained_rouge = calculate_rouge_score(untrained_output, reference)

    print(f"Original: {example}")
    print(f"Reference: {reference}")
    print(f"Trained Model Output: {trained_output}")
    print(f"Trained BLEU Score: {trained_bleu:.4f}")
    print(f"Trained ROUGE Scores: {trained_rouge}")
    print(f"Untrained Model Output: {untrained_output}")
    print(f"Untrained BLEU Score: {untrained_bleu:.4f}")
    print(f"Untrained ROUGE Scores: {untrained_rouge}")
    print("-----------------------------------------------------")


-----------------------------------------------------
Original: Who is better leader male or female
Reference: Anyone can lead teams, regardless of gender.
Trained Model Output: Anyone can lead teams, regardless of gender.
Trained BLEU Score: 1.0000
Trained ROUGE Scores: {'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rouge2': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
Untrained Model Output: Make this sentence fair and unbiased: Who is better leader male or female?
Untrained BLEU Score: 0.0000
Untrained ROUGE Scores: {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.0, recall=0.0, fmeasure=0.0)}
-----------------------------------------------------
-----------------------------------------------------
Original: Who is better in sports boys or girls
Reference: Anyone can excel in sports, regardless of gender.
Trained M

/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Original: older workers shouldn't work in education
Reference: Success in education depends on effort, not gender
Trained Model Output: Anyone with skills can manage high-pressure jobs, regardless of disability.
Trained BLEU Score: 0.0000
Trained ROUGE Scores: {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.0, recall=0.0, fmeasure=0.0)}
Untrained Model Output: Make this sentence fair and unbiased: older workers shouldn't work in education: older workers shouldn't work in education: older workers shouldn't work in education: older workers shouldn't work in education: older workers shouldn't work
Untrained BLEU Score: 0.0000
Untrained ROUGE Scores: {'rouge1': Score(precision=0.05128205128205128, recall=0.25, fmeasure=0.0851063829787234), 'rouge2': Score(precision=0.02631578947368421, recall=0.14285714285714285, fmeasure=0.044444444444444446), 'rougeL': Score(precision=0.05128205128205128, rec

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the saved model and tokenizer
model_path = "./t5_debias_model_v2"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

# Function to evaluate the model
def evaluate_model(input_text):
    # Add the prompt to the input text
    input_prompt = "Rewrite the following sentence to remove bias: " + input_text
    # Tokenize the input
    inputs = tokenizer(
        input_prompt,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128
    )
    # Generate the output
    outputs = model.generate(inputs["input_ids"], max_new_tokens=50)
    # Decode and return the output
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Updated test examples (selected from the biased dataset)
test_examples = [
    "Who is better leader male or female",
    "Who is better in sports boys or girls",
    "It's rare for people with disabilities to achieve greatness in STEM."
]

# Test the model with updated examples
for example in test_examples:
    print(f"Original: {example}")
    print(f"Debiased: {evaluate_model(example)}")
    print()
